In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mndanish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mndanish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
tweets = pd.read_csv('Tweet.csv')
company_tweet = pd.read_csv('Company_Tweet.csv')

tweets = pd.merge(tweets,company_tweet,on='tweet_id',how='inner')

In [56]:
tweets['post_date'] = pd.to_datetime(tweets['post_date'], unit='s')
tweets['date'] = pd.to_datetime(tweets['post_date'].apply(lambda date: date.date()))

In [57]:
tweets.drop(columns = ['tweet_id', 'writer', 'post_date', 'comment_num', 'retweet_num', 'like_num'], inplace = True)

In [58]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [59]:
tweets['body'] = tweets['body'].str.lower()
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r'{link}', '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r"\[video\]", '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r'@mention', '', x))
tweets['body'] = tweets['body'].apply(lambda x: re.sub(r'\B#\w\w+', '', x))
tweets['body'] = tweets['body'].apply(lambda x: remove_stopwords(x))

In [60]:
for i in range(len(tweets)):
    sentence = tweets.at[i, 'body']
    sentiment = SentimentIntensityAnalyzer().polarity_scores(sentence)
    tweets.at[i, 'sentiment_score'] = sentiment['compound']

In [61]:
tweets.to_csv('tweets_sentiment.csv', index=False)